# Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.\
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.\
#### Описание данных
children — количество детей в семье\
days_employed — общий трудовой стаж в днях\
dob_years — возраст клиента в годах\
education — уровень образования клиента\
education_id — идентификатор уровня образования\
family_status — семейное положение\
family_status_id — идентификатор семейного положения\
gender — пол клиента\
income_type — тип занятости\
debt — имел ли задолженность по возврату кредитов\
total_income — ежемесячный доход\
purpose — цель получения кредита\
## Инструкция по выполнению
### Шаг 1. Откройте таблицу и изучите общую информацию о данных

In [1]:
import pandas as pd
import numpy as np
import pprint
from pymystem3 import Mystem
from collections import Counter
pd.set_option('precision', 2) # Два знака после запятой

In [2]:
df = pd.read_csv('data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу
5,0,-926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,покупка жилья
6,0,-2879.20,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,операции с жильем
7,0,-152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.93,образование
8,2,-6929.87,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.83,на проведение свадьбы
9,0,-2188.76,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.describe()
# df.purpose.unique()
# df.family_status.unique()
# df.education.unique()
# df.gender.unique() # Есть одна запись с чем-то непонятным
# df[df['gender'] == 'XNA'].count()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.00,19351.00,21525.00,21525.00,21525.00,21525.00,1.94e+04
mean,0.54,63046.50,43.29,0.82,0.97,0.08,1.67e+05
std,1.38,140827.31,12.57,0.55,1.42,0.27,1.03e+05
min,-1.00,-18388.95,0.00,0.00,0.00,0.00,2.07e+04
25%,0.00,-2747.42,33.00,1.00,0.00,0.00,1.03e+05
50%,0.00,-1203.37,42.00,1.00,0.00,0.00,1.45e+05
75%,1.00,-291.10,53.00,1.00,1.00,0.00,2.03e+05
max,20.00,401755.40,75.00,4.00,4.00,1.00,2.27e+06


##### Выводы
1 Пропущенные значения встречаются в колонках days_employed (стаж) и total_income (доход).\
2 Отрицательные значения в колонках children и days_employed\
3 Некорректные типы данных days_employed\
4 Неудорбные данные в total_income (доход)\
5 Разный регистр в столбце education\
6 Дублирующие признаки в обзовании и id, и также в family ststus\
7 Дни стажа логичнее перевести в года для удобства восприятия\
8 20 детей очень мало вероятно, скорее всего выброс

### <a name="contents">Шаг 2. Предобработка данных</a>
#### [1 определите и заполните пропущенные значения:](#stage_1)
    опишите, какие пропущенные значения вы обнаружили;
    приведите возможные причины появления пропусков в данных;
    объясните, по какому принципу заполнены пропуски;

#### [2 замените вещественный тип данных на целочисленный:](#stage_2)
    поясните, как выбирали метод для изменения типа данных;

#### [3 удалите дубликаты:](#stage_3)
    поясните, как выбирали метод для поиска и удаления дубликатов в данных;
    приведите возможные причины появления дубликатов;

#### [4 выделите леммы в значениях столбца с целями получения кредита:](#stage_4)
    опишите, как вы проводили лемматизацию целей кредита;

#### [5 категоризируйте данные:](#stage_5)
    перечислите, какие «словари» вы выделили для этого набора данных, и объясните, почему.

<a name="stage_1">1 Определить пропущенные значения</a>\
[Наверх к оглавлению](#contents)\

In [5]:
df[ df.days_employed.isna() ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Пропуски в двух столбцах одновременно. Это может быть связано с тем, что часть клиентов не работали и соответственно не получали зарплату\
Для ответов на поставленные в исследовани вопросы, нужно внимательнее изучить и скорретировать столбцы:\
children\
family_status_id\
total_income\
purpose\
debt\
Приступим

Начнем по порядку с children.

In [6]:
#Посмотрим уникальные значения
df['children'].unique().tolist()

[1, 0, 3, 2, -1, 4, 20, 5]

In [7]:
print("Заявок с значением -1 в столбце дети:", df[ df['children'] == -1 ].count()[0])
print("Заявок с значением 20 в столбце дети:", df[ df['children'] == 20 ].count()[0])
# df.nunique(dropna=False) # Считать кол-во вместе с NaN
# print("Всего заявок со значением 20 в столбце дети:", df[ df['children'] == 20 ]['total_income'].nunique(dropna=False))

Заявок с значением -1 в столбце дети: 47
Заявок с значением 20 в столбце дети: 76


76 строк с 20 детьми похоже на человеческую ошибку, изменим это значение на 2.\
a 47 строк с отрицательным значением -1 на 1.

In [8]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)
#Посмотрим уникальные значения
df['children'].unique().tolist()

[1, 0, 3, 2, 4, 5]

Данные по детям обработаны, продолжин с графой о семейном положении.

In [9]:
#Посмотрим уникальные значения 
df['family_status'].unique().tolist()

['женат / замужем',
 'гражданский брак',
 'вдовец / вдова',
 'в разводе',
 'Не женат / не замужем']

In [10]:
df['family_status_id'].unique().tolist()

[0, 1, 2, 3, 4]

In [11]:
# Приведение к нижнему регистру
df.education = df.education.str.lower()
# df.purpose = df.purpose.str.lower()

Перейдём к столбцу Доход.
Как помним, у нас там есть значения NaN. Мы предположили, что это может быть связано с тем, что часть клиентов не работали и соответственно не получали зарплату. Проверим нашу гипотезу.

In [12]:
# Пропущенные значения должны быть в двух столбцах в одной строке
# df[ df.days_employed.isna() ] # 2174 строки
df[ (df.days_employed.isna()) & (df.total_income.isna()) ] # Тоже 2174 строки

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Гипотеза подтвердилась. Данные не заполнены в 2 столбцах одновременно.

In [13]:
df[ df.days_employed.isna() ]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Корректно будет заменить пропущенные значения дохода на 0.

In [14]:
df['total_income'] = df['total_income'].fillna(0)
df['days_employed'] = df['days_employed'].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [15]:
# Проверим стаж в днях, кажется очень большие значения. Переведём в года.
df['days_employed'].describe()

count     21525.00
mean      56678.87
std      134870.76
min      -18388.95
25%       -2518.17
50%        -982.53
75%           0.00
max      401755.40
Name: days_employed, dtype: float64

In [16]:
df['years_employed'] = df['days_employed'] / 365
# df[['days_employed', 'years_employed']]
df[['days_employed', 'years_employed']].describe()
# Явно присутствую нереальные значения. Невозможно иметь стаж больше 100 лет. 

,days_employed,years_employed
count,21525.00,21525.00
mean,56678.87,155.28
std,134870.76,369.51
min,-18388.95,-50.38
25%,-2518.17,-6.90
50%,-982.53,-2.69
75%,0.00,0.00
max,401755.40,1100.70


In [17]:
# Найдем самого великовозрастного человека в наборе
# Если предположить, что он работал с 16 лет, то:
max_years_of_employment = (df['dob_years'].max() - 16)
max_years_of_employment # Получим количество лет, по которым можно сделать отбор данных

59

In [18]:
df[ df.years_employed > 80 ]
# Довольно много строк 3445 из 21525 со стажем более 80 лет. Следует спросить про это у Банка!

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,932.24
18,0,400281.14,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.78,на покупку подержанного автомобиля,1096.66
24,1,338551.95,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.24,операции с коммерческой недвижимостью,927.54
25,0,363548.49,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.76,покупка недвижимости,996.02
30,1,335581.67,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.07,операции с коммерческой недвижимостью,919.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.87,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.99,сыграть свадьбу,928.51
21508,0,386497.71,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.59,недвижимость,1058.90
21509,0,362161.05,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.06,операции с недвижимостью,992.22
21518,0,373995.71,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864.65,сделка с автомобилем,1024.65


<a name="stage_2">2 Зменить вещественный тип данных на целочисленный</a>\
[Наверх к оглавлению](#contents)\

days_employed заменим на целочисленный тип и приведём к году

In [19]:
df.days_employed = df.days_employed.astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int32  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  years_employed    21525 non-null  float64
dtypes: float64(2), int32(1), int64(5), object(5)
memory usage: 2.1+ MB


Заменим отрицательные значения в стаже. Причина их появления неясна, на это нужно обратить внимание сотрудников банка\
Исправим это

In [20]:
df.days_employed = df.days_employed.abs()

In [21]:
df.tail(10)
# df.describe()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
21515,1,467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.33,заняться образованием,-1.28
21516,0,914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.78,покупка своего жилья,-2.51
21517,0,404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.55,на покупку своего автомобиля,-1.11
21518,0,373995,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864.65,сделка с автомобилем,1024.65
21519,1,2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.04,покупка коммерческой недвижимости,-6.44
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.86,операции с жильем,-12.41
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.81,сделка с автомобилем,942.29
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.56,недвижимость,-5.79
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.05,на покупку своего автомобиля,-8.53
21524,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.42,на покупку автомобиля,-5.44


<a name="stage_3">3 Удаление дубликатов</a>\
[Наверх к оглавлению](#contents)\

Подсчитаем кол-во дупликатов

In [22]:
print('Дубликатов в таблице: ')
df.duplicated().sum()

Дубликатов в таблице: 


71

In [23]:
# Посмотрим, как они выглядят
df[ df.duplicated() ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,покупка жилья для семьи,0.0
3290,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,0.0,сыграть свадьбу,0.0
4182,1,0,34,высшее,0,гражданский брак,1,F,сотрудник,0,0.0,свадьба,0.0
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,0.0,свадьба,0.0
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,0.0,сыграть свадьбу,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,0,64,среднее,1,женат / замужем,0,F,пенсионер,0,0.0,дополнительное образование,0.0
21032,0,0,60,среднее,1,женат / замужем,0,F,пенсионер,0,0.0,заняться образованием,0.0
21132,0,0,47,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,ремонт жилью,0.0
21281,1,0,30,высшее,0,женат / замужем,0,F,сотрудник,0,0.0,покупка коммерческой недвижимости,0.0


In [24]:
df.drop_duplicates(inplace=True)
print('Дубликатов в таблице: ')
df.duplicated().sum()

Дубликатов в таблице: 


0

##### Что можно улучшить:
Опиши возможные причины дублей в заявках на кредит. На работе ты можешь это использовать при предобработке подобных датасетов, или, еще лучше, изменить процесс заполнения заявок.

<a name="stage_4">4 Выделение леммы в значениях столбца с целями получения кредита</a>\
[Наверх к оглавлению](#contents)

Для указания цели кредита не было единого списка целей, поэтому люди писали своими словами. Для анализа возвратности по целям кредита, нам необходимо проанализировать ответы в заявках и собрать их в общие по целям группы.\
Приступаем к выявлению лемм.

In [25]:
#Посмотрим что и в каком количестве писали клиенты о своих целях кредита 
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [26]:
unique_credit_purposes = df['purpose'].unique().tolist()
m = Mystem()
str = '; '.join(unique_credit_purposes) # Разделение строк ";"
pprint.pprint(str) # Вывод

('покупка жилья; приобретение автомобиля; дополнительное образование; сыграть '
 'свадьбу; операции с жильем; образование; на проведение свадьбы; покупка '
 'жилья для семьи; покупка недвижимости; покупка коммерческой недвижимости; '
 'покупка жилой недвижимости; строительство собственной недвижимости; '
 'недвижимость; строительство недвижимости; на покупку подержанного '
 'автомобиля; на покупку своего автомобиля; операции с коммерческой '
 'недвижимостью; строительство жилой недвижимости; жилье; операции со своей '
 'недвижимостью; автомобили; заняться образованием; сделка с подержанным '
 'автомобилем; получение образования; автомобиль; свадьба; получение '
 'дополнительного образования; покупка своего жилья; операции с недвижимостью; '
 'получение высшего образования; свой автомобиль; сделка с автомобилем; '
 'профильное образование; высшее образование; покупка жилья для сдачи; на '
 'покупку автомобиля; ремонт жилью; заняться высшим образованием')


In [27]:
lemmas = m.lemmatize(str)
# type(lemmas) # list
pprint.pprint(Counter(lemmas))

unique_lemmas = Counter(lemmas)
# unique_lemmas
# sorted(lemmas.items(), key = lambda pair: pair[1], reverse=True)

Counter({' ': 59,
         '; ': 37,
         'покупка': 10,
         'недвижимость': 10,
         'автомобиль': 9,
         'образование': 9,
         'жилье': 7,
         'с': 5,
         'операция': 4,
         'на': 4,
         'свой': 4,
         'свадьба': 3,
         'строительство': 3,
         'получение': 3,
         'высокий': 3,
         'дополнительный': 2,
         'для': 2,
         'коммерческий': 2,
         'жилой': 2,
         'подержать': 2,
         'заниматься': 2,
         'сделка': 2,
         'приобретение': 1,
         'сыграть': 1,
         'проведение': 1,
         'семья': 1,
         'собственный': 1,
         'со': 1,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})


In [28]:
# Напишем функцию для лемматизации целей кредита
# Долго грузит
# def lemma_group(purpose):
#     lemmas = ''.join(m.lemmatize(purpose))
#     if 'автомоб' in lemmas:
#         return 'покупка автомобиля'
#     elif ('недвиж' in lemmas) or ('жиль' in lemmas):
#         return 'покупка недвижимости'
#     elif 'свадьб' in lemmas:
#         return 'свадьба'
#     elif 'образован' in lemmas:
#         return 'образование'
#     else:
#         return 'другая цель кредита'

# # Применяем функцию лемматизации к столбцу purpose и добавляем новый столбец с леммами purpose_category
# df['purpose_group'] = df['purpose'].apply(lemma_group)
# df.head(10)

In [29]:
#Посмотрим количество заявок по каждой цели кредита
# purposes = df.groupby('purpose_group')['purpose_group'].count().sort_values(ascending = False)

##### Выводы
С помощью лемматизации нам удалось выделить из общей массы ответов четкий список, состоящий из 4 основных целей кредита. Полученные данные необходимы нам для дальнейшего анализа возвратности по целям кредита.\
Как видим, почти половина заемщиков планирует приобрести недвижиость, другие автомобиль или получить образование.

<a name="stage_5">5 Категоризация данных</a>\
[Наверх к оглавлению](#contents)\

Для удобства исследования необходимо разбить данные о доходах на группы, желательно на равные группы, чтобы было репрезентативно.\
Разделить данные на сегменты одинакового размера поможет функция .qcut()\
В аргументе укажем значение 5. Группировка на 4-5 значений в нашем случае будет оптимальна.\

In [30]:
income_cut = pd.qcut(df['total_income'], 5)
income_cut.value_counts()

(214606.464, 2265604.029]    4291
(156426.09, 214606.464]      4291
(79166.246, 116408.006]      4291
(-0.001, 79166.246]          4291
(116408.006, 156426.09]      4290
Name: total_income, dtype: int64

Пороговые значения определены, можно написать функцию категоризации и добавить новый столбец.

In [31]:
def income_group(total_income):
    if total_income < 79166.2:
        return('низкие доходы')
    elif 79166.2 <= total_income < 116408:
        return('средние доходы')
    elif 116408 <= total_income <= 156426:
        return('повышенные доходы')
    elif 156426 <= total_income <= 214606.4:
        return('высокие доходы')
    return('сверхвысокие доходы')

In [32]:
df['income_group'] = df['total_income'].apply(income_group)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,-23.12,сверхвысокие доходы
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,-11.03,средние доходы
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,-15.41,повышенные доходы
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,-11.30,сверхвысокие доходы
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,932.24,высокие доходы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.86,операции с жильем,-12.41,сверхвысокие доходы
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.81,сделка с автомобилем,942.29,повышенные доходы
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.56,недвижимость,-5.79,средние доходы
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.05,на покупку своего автомобиля,-8.53,сверхвысокие доходы


In [33]:
def children_group(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетный клиент'
# Добавить столбец
df['children_group'] = df.apply(children_group, axis=1)

##### Шаг 3. Ответьте на вопросы:
Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
# Есть ли зависимость между наличием детей и возвратом кредита в срок?
# df[ df.debt == 1 ].count()[0] # 1741 Сколько должников
not_children_ok = df[ (df.debt == 1) & (df.children == 0)].count()[0] # 1063 Нет детей и задолженность
not_children_all = df[ df.children == 0].count()[0] # 14091 Нет детей и вернул в срок
list = []
list.append(not_children_ok / not_children_all)
list

[0.07543822297920659]

In [35]:
pivot_table_children = df.pivot_table(index='children_group', columns='debt', values='children', aggfunc='count')
pivot_table_children['percent'] = pivot_table_children[1] / (pivot_table_children[0] + pivot_table_children[1]) * 100
pivot_table_children.columns = ['Вернули в срок','С задолженностью','% Должников']
pivot_table_children

,Вернули в срок,С задолженностью,% Должников
children_group,,,
1-2 ребенка,6336,647,9.27
многодетный клиент,349,31,8.16
нет детей,13028,1063,7.54


##### Вывод
Клиенты без детей менее склонны к просрочке по выплатам кредита.

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
pivot_table_family_status = df.pivot_table(index='family_status', columns='debt', values='children', aggfunc='count')
pivot_table_family_status['percent'] = pivot_table_family_status[1] / (pivot_table_family_status[0] + pivot_table_family_status[1]) * 100
pivot_table_family_status.columns = ['Вернули в срок','С задолженностью','% Должников']
pivot_table_family_status.sort_values(by = '% Должников', ascending = False)

,Вернули в срок,С задолженностью,% Должников
family_status,,,
Не женат / не замужем,2536,274,9.75
гражданский брак,3763,388,9.35
женат / замужем,11408,931,7.55
в разводе,1110,85,7.11
вдовец / вдова,896,63,6.57


##### Вывод
Зависимость между семейным положением и возвратом кредита в срок есть. Клиенты не состоящие и не состояышие ранее в браке более склонны к наличию задолженности по возврату кредита. Клиенты находящиеся в разводе и овдовевшие имеют меньше % задолженностей перед банком.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [37]:
pivot_table_income = df.pivot_table(index='income_group', columns='debt', values='children', aggfunc='count')
pivot_table_income['percent'] = pivot_table_income[1] / (pivot_table_income[0] + pivot_table_income[1]) * 100
pivot_table_income.columns = ['Вернули в срок','С задолженностью','% Должников']
pivot_table_income.sort_values(by = '% Должников', ascending = False)

,Вернули в срок,С задолженностью,% Должников
income_group,,,
повышенные доходы,3915,375,8.74
высокие доходы,3921,370,8.62
средние доходы,3928,363,8.46
низкие доходы,3958,333,7.76
сверхвысокие доходы,3991,300,6.99


##### Вывод
Клиенты со сверхвысоким уровнем доходов менее склонны иметь просрочки по кредитам.
У остальных групп примерно одинаковые значения % наличия просрочки, но среди них клиенты с самыми низкими из рассматриваемых доходами имеют меньше % задолженности по возврату кредитов

Как разные цели кредита влияют на его возврат в срок?

In [38]:
# Попробуем через map(), составим словарь из двух списков
credit_purpose_keys= []
credit_purpose_list = df.purpose.unique().tolist()
for value in range(len(credit_purpose_list)):
    if 'образован' in credit_purpose_list[value]:
        credit_purpose_keys.append('Образование')
    elif 'авто' in credit_purpose_list[value]:
        credit_purpose_keys.append('Автомобиль')
    elif 'свадь' in credit_purpose_list[value]:
        credit_purpose_keys.append('Свадьба')
    else:
        credit_purpose_keys.append('Недвижимость')
        
credit_purpose_keys


['Недвижимость',
 'Автомобиль',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Автомобиль',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Свадьба',
 'Образование',
 'Недвижимость',
 'Недвижимость',
 'Образование',
 'Автомобиль',
 'Автомобиль',
 'Образование',
 'Образование',
 'Недвижимость',
 'Автомобиль',
 'Недвижимость',
 'Образование']

In [39]:
credit_purpose_list

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [40]:
# Сшили словарь, теперь можно передать в map()
credit_purpose_dict = dict(zip(credit_purpose_list, credit_purpose_keys))

# Готово, работает, хотя не уверен, что нужно было действовать именно так
df['credit_purpose_status'] = df['purpose'].map(credit_purpose_dict)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,income_group,children_group,credit_purpose_status
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,-23.12,сверхвысокие доходы,1-2 ребенка,Недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,-11.03,средние доходы,1-2 ребенка,Автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,-15.41,повышенные доходы,нет детей,Недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,-11.30,сверхвысокие доходы,многодетный клиент,Образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,932.24,высокие доходы,нет детей,Свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.86,операции с жильем,-12.41,сверхвысокие доходы,1-2 ребенка,Недвижимость
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.81,сделка с автомобилем,942.29,повышенные доходы,нет детей,Автомобиль
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.56,недвижимость,-5.79,средние доходы,1-2 ребенка,Недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.05,на покупку своего автомобиля,-8.53,сверхвысокие доходы,многодетный клиент,Автомобиль


In [41]:
pivot_table_credit_purpose = df.pivot_table(index='credit_purpose_status', columns='debt', values='children', aggfunc='count')
pivot_table_credit_purpose['percent'] = pivot_table_credit_purpose[1] / (pivot_table_credit_purpose[0] + pivot_table_credit_purpose[1]) * 100
pivot_table_credit_purpose.columns = ['Вернули в срок','С задолженностью','% Должников']
pivot_table_credit_purpose.sort_values(by = '% Должников', ascending = False)

,Вернули в срок,С задолженностью,% Должников
credit_purpose_status,,,
Автомобиль,3903,403,9.36
Образование,3643,370,9.22
Свадьба,2138,186,8.00
Недвижимость,10029,782,7.23


##### Вывод
Клиенты, взявшие кредит на покупку недвижимости и проведение свадьбы менее склонны иметь задолженность по выплатам перед банком.

Шаг 4. Напишите общий вывод

Есть зависимости между целями кредита, семейным положением, уровнем дохода и количеством детей в семье. Нельзя сказать, что они кардинально отличаются, но 2% на больших числах могут оказать существенное влияние на кредитный портфель банка. Особое внимание следует уделить вдовам/вдовцам или молодоженам — с ними могут риски по невозврату кредитов выше.

##### Рекомендации для банка
- Уделить внимание правилам валидации на сервере, чтобы больше не сталкиваться с проблемой дублей.
- Продумать сценарий, когда отличается дата или время заявки, а уникальный идентификатор пользователя один и тот же, 
  иначе фактически дубли будут, но метод duplicated() их не найдет.
- Считать стаж работы в годах, пользователи чаще всего не знают свой стаж в днях.
- Показывать операторам в банке или пользователям на сайте подсказки заранее, когда вводят сверхвысокие/сверхнизкие
  значения в полях: возраст, стаж, количество детей.
- Блокировать отправку заявок с заведомо некорректными значениями, которые могут быть важны для анализа, например,
  с отрицательным количеством детей.